In [1]:
import numpy as np

def vernam_operation(src_array,keystream):
    key_len = len(keystream)
    return bytearray([src_array[i] ^^ keystream[i % key_len] for i in range(len(src_array))])

# Convert bytearray to sth like FF FA 01 BB
def bytearray_to_readable(btarr):
    return " ".join([hex(b)[2:].upper().zfill(2) for b in btarr])

# All parameters involving plaintexts or ciphertexts assume bytearray
class Vernam:
    def __init__(self,keystream):
        self.keystream = keystream
    @staticmethod
    def keygen(length):
        gen = np.random.default_rng()
        keystream = bytearray()
        bitlen = 2 ** 8
        for _ in range(length):
            keystream.append(gen.integers(0,bitlen))
        return keystream
    def operation(self,a):
        return vernam_operation(a,self.keystream)
    # Reverse of XOR is also XOR
    # Creating shorcuts for readibility
    encrypt = operation
    decrypt = operation
    # Recover key from plain XOR crypt
    @staticmethod
    def recover_key(plain,crypt):
        assert len(plain) == len(crypt)
        return bytearray([plain[i] ^^ crypt[i] for i in range(len(plain))])

In [2]:
key = Vernam.keygen(10) # Generate a fixed-length key, Less secure than OTP
print(bytearray_to_readable(key))

0B D1 FA 23 18 B6 E0 55 47 A7


In [3]:
ve = Vernam(key)
m = "We are discovered, safe yourself."
m_btarr = bytearray(m,encoding="ascii")
c = ve.encrypt(m_btarr)
rtn_m_btarr = ve.decrypt(c)
rtn_m = rtn_m_btarr.decode(encoding="ascii")

print(m,bytearray_to_readable(m_btarr))
print(bytearray_to_readable(c))
print(rtn_m,bytearray_to_readable(rtn_m_btarr))
print(bytearray_to_readable(Vernam.recover_key(m_btarr,c)))

We are discovered, safe yourself. 57 65 20 61 72 65 20 64 69 73 63 6F 76 65 72 65 64 2C 20 73 61 66 65 20 79 6F 75 72 73 65 6C 66 2E
5C B4 DA 42 6A D3 C0 31 2E D4 68 BE 8C 46 6A D3 84 79 67 D4 6A B7 9F 03 61 D9 95 27 34 C2 67 B7 D4
We are discovered, safe yourself. 57 65 20 61 72 65 20 64 69 73 63 6F 76 65 72 65 64 2C 20 73 61 66 65 20 79 6F 75 72 73 65 6C 66 2E
0B D1 FA 23 18 B6 E0 55 47 A7 0B D1 FA 23 18 B6 E0 55 47 A7 0B D1 FA 23 18 B6 E0 55 47 A7 0B D1 FA


## One-time Pad (OTP)

In [4]:
def OTP_encrypt(m):
    key = Vernam.keygen(len(m))
    return Vernam(key).encrypt(m), key

def OTP_decrypt(c,key):
    return Vernam(key).decrypt(c)

for i in range(5):
    print("Run {}".format(i + 1))
    m = "We are discovered, safe yourself."
    m_btarr = bytearray(m,encoding="ascii")
    c, key = OTP_encrypt(m_btarr)
    rtn_m_btarr = OTP_decrypt(c, key)
    rtn_m = rtn_m_btarr.decode(encoding="ascii")

    print(bytearray_to_readable(key))
    print(m,bytearray_to_readable(m_btarr))
    print(bytearray_to_readable(c))
    print(rtn_m,bytearray_to_readable(rtn_m_btarr))

Run 1
E2 88 E9 BD 1C FE F5 FB 6E 53 63 F3 9E E1 19 94 66 C1 50 42 52 65 4C FD CC 74 A0 E7 D4 E0 0F 56 CA
We are discovered, safe yourself. 57 65 20 61 72 65 20 64 69 73 63 6F 76 65 72 65 64 2C 20 73 61 66 65 20 79 6F 75 72 73 65 6C 66 2E
B5 ED C9 DC 6E 9B D5 9F 07 20 00 9C E8 84 6B F1 02 ED 70 31 33 03 29 DD B5 1B D5 95 A7 85 63 30 E4
We are discovered, safe yourself. 57 65 20 61 72 65 20 64 69 73 63 6F 76 65 72 65 64 2C 20 73 61 66 65 20 79 6F 75 72 73 65 6C 66 2E
Run 2
38 DE 2C A3 7C B4 65 94 B6 A5 70 2C 77 84 C9 54 97 A4 8D D2 8B 8F 94 CF 58 F3 66 0F 3D A1 40 6C 68
We are discovered, safe yourself. 57 65 20 61 72 65 20 64 69 73 63 6F 76 65 72 65 64 2C 20 73 61 66 65 20 79 6F 75 72 73 65 6C 66 2E
6F BB 0C C2 0E D1 45 F0 DF D6 13 43 01 E1 BB 31 F3 88 AD A1 EA E9 F1 EF 21 9C 13 7D 4E C4 2C 0A 46
We are discovered, safe yourself. 57 65 20 61 72 65 20 64 69 73 63 6F 76 65 72 65 64 2C 20 73 61 66 65 20 79 6F 75 72 73 65 6C 66 2E
Run 3
10 21 35 D4 AB EA 2E DC 2B F5 8B 5B B5 76 38 85 7E 22 